### CSV And EXcel files- Structured Data

In [1]:
import pandas as pd

In [2]:
# Create sample data
data = {
    'Product': ['Laptop', 'Mouse', 'Keyboard', 'Monitor', 'Webcam'],
    'Category': ['Electronics', 'Accessories', 'Accessories', 'Electronics', 'Electronics'],
    'Price': [999.99, 29.99, 79.99, 299.99, 89.99],
    'Stock': [50, 200, 150, 75, 100],
    'Description': [
        'High-performance laptop with 16GB RAM and 512GB SSD',
        'Wireless optical mouse with ergonomic design',
        'Mechanical keyboard with RGB backlighting',
        '27-inch 4K monitor with HDR support',
        '1080p webcam with noise cancellation'
    ]
}

# Save as CSV
df = pd.DataFrame(data)
df.to_csv('data/structured_file/products.csv', index=False)
df

,Product,Category,Price,Stock,Description
0,Laptop,Electronics,999.99,50,High-performance laptop with 16GB RAM and 512G...
1,Mouse,Accessories,29.99,200,Wireless optical mouse with ergonomic design
2,Keyboard,Accessories,79.99,150,Mechanical keyboard with RGB backlighting
3,Monitor,Electronics,299.99,75,27-inch 4K monitor with HDR support
4,Webcam,Electronics,89.99,100,1080p webcam with noise cancellation


In [3]:
# Save as Excel with multiple sheets
with pd.ExcelWriter('data/structured_file/inventoryy.xlsx') as writer:
    df.to_excel(writer, sheet_name='Products', index=False)
    
    # Add another sheet
    summary_data = {
        'Category': ['Electronics', 'Accessories'],
        'Total_Items': [3, 2],
        'Total_Value': [1389.97, 109.98]
    }
    pd.DataFrame(summary_data).to_excel(writer, sheet_name='Summary', index=False)

## CSV Processing

In [4]:
from langchain_community.document_loaders import CSVLoader
from langchain_community.document_loaders import UnstructuredCSVLoader

In [5]:
# Method 1: CSVLoader - Each row becomes a document
print("1️⃣ CSVLoader - Row-based Documents")
csv_loader = CSVLoader(
    file_path='data/structured_file/products.csv',
    encoding='utf-8',
    csv_args={
        'delimiter': ',',
        'quotechar': '"',
    }
)

csv_docs = csv_loader.load()
print(csv_docs)
print(f"Loaded {len(csv_docs)} documents (one per row)")
print("\nFirst document:")
print(f"Content: {csv_docs[0].page_content}")
print(f"Metadata: {csv_docs[0].metadata}")

1️⃣ CSVLoader - Row-based Documents
[Document(metadata={'source': 'data/structured_file/products.csv', 'row': 0}, page_content='Product: Laptop\nCategory: Electronics\nPrice: 999.99\nStock: 50\nDescription: High-performance laptop with 16GB RAM and 512GB SSD'), Document(metadata={'source': 'data/structured_file/products.csv', 'row': 1}, page_content='Product: Mouse\nCategory: Accessories\nPrice: 29.99\nStock: 200\nDescription: Wireless optical mouse with ergonomic design'), Document(metadata={'source': 'data/structured_file/products.csv', 'row': 2}, page_content='Product: Keyboard\nCategory: Accessories\nPrice: 79.99\nStock: 150\nDescription: Mechanical keyboard with RGB backlighting'), Document(metadata={'source': 'data/structured_file/products.csv', 'row': 3}, page_content='Product: Monitor\nCategory: Electronics\nPrice: 299.99\nStock: 75\nDescription: 27-inch 4K monitor with HDR support'), Document(metadata={'source': 'data/structured_file/products.csv', 'row': 4}, page_content='Pro

In [6]:
from typing import List
from langchain_core.documents import Document

# Method 2: Custom CSV processing for better control
print("\n2️⃣ Custom CSV Processing")
def process_csv_intelligently(filepath: str) -> List[Document]:
    """Process CSV with intelligent document creation"""
    df = pd.read_csv(filepath)
    documents = []
    
    # Strategy 1: One document per row with structured content
    for idx, row in df.iterrows():
        # Create structured content
        content = f"""Product Information:
        Name: {row['Product']}
        Category: {row['Category']}
        Price: ${row['Price']}
        Stock: {row['Stock']} units
        Description: {row['Description']}"""
        
        # Create document with rich metadata
        doc = Document(
            page_content=content,
            metadata={
                'source': filepath,
                'row_index': idx,
                'product_name': row['Product'],
                'category': row['Category'],
                'price': row['Price'],
                'data_type': 'product_info'
            }
        )
        documents.append(doc)
    return documents


2️⃣ Custom CSV Processing


In [7]:
process_csv_intelligently('data/structured_file/products.csv')

[Document(metadata={'source': 'data/structured_file/products.csv', 'row_index': 0, 'product_name': 'Laptop', 'category': 'Electronics', 'price': 999.99, 'data_type': 'product_info'}, page_content='Product Information:\n        Name: Laptop\n        Category: Electronics\n        Price: $999.99\n        Stock: 50 units\n        Description: High-performance laptop with 16GB RAM and 512GB SSD'),
 Document(metadata={'source': 'data/structured_file/products.csv', 'row_index': 1, 'product_name': 'Mouse', 'category': 'Accessories', 'price': 29.99, 'data_type': 'product_info'}, page_content='Product Information:\n        Name: Mouse\n        Category: Accessories\n        Price: $29.99\n        Stock: 200 units\n        Description: Wireless optical mouse with ergonomic design'),
 Document(metadata={'source': 'data/structured_file/products.csv', 'row_index': 2, 'product_name': 'Keyboard', 'category': 'Accessories', 'price': 79.99, 'data_type': 'product_info'}, page_content='Product Informatio

In [8]:
# 📊 CSV Processing Strategies
print("\n📊 CSV Processing Strategies:")
print("\n1. Row-based (CSVLoader):")
print("  ✅ Simple one-row-one-document")
print("  ✅ Good for record lookups")
print("  ❌ Loses table context")

print("\n2. Intelligent Processing:")
print("  ✅ Preserves relationships")
print("  ✅ Creates summaries")
print("  ✅ Rich metadata")
print("  ✅ Better for Q&A")


📊 CSV Processing Strategies:

1. Row-based (CSVLoader):
  ✅ Simple one-row-one-document
  ✅ Good for record lookups
  ❌ Loses table context

2. Intelligent Processing:
  ✅ Preserves relationships
  ✅ Creates summaries
  ✅ Rich metadata
  ✅ Better for Q&A


### Excel Processing

In [9]:
def process_excel_with_pandas(filepath: str):
    documents = []

    excel_file = pd.ExcelFile(filepath, engine="openpyxl")

    for sheet_name in excel_file.sheet_names:
        df = pd.read_excel(
            filepath,
            sheet_name=sheet_name,
            engine="openpyxl"
        )

        content = f"Sheet: {sheet_name}\n"
        content += f"Columns: {', '.join(map(str, df.columns))}\n"
        content += f"Rows: {len(df)}\n\n"
        content += df.to_markdown(index=False)

        documents.append(
            Document(
                page_content=content,
                metadata={
                    "source": filepath,
                    "sheet": sheet_name,
                    "rows": len(df),
                    "cols": len(df.columns)
                }
            )
        )

    return documents


In [10]:
excel_docs = process_excel_with_pandas('data/structured_file/inventory.xlsx')
print(f"Processed {len(excel_docs)} sheets")

SyntaxError: invalid syntax (__init__.py, line 1187)

In [25]:
from langchain_community.document_loaders import UnstructuredExcelLoader
# Method 2: UnstructuredExcelLoader
print("\n2️⃣ UnstructuredExcelLoader")
try:
    excel_loader = UnstructuredExcelLoader(
        'data/structured_files/inventory.xlsx',
        mode="elements"
    )
    unstructured_docs = excel_loader.load()
    print("  ✅ Handles complex Excel features")
    print("  ✅ Preserves formatting info")
    print("  ❌ Requires unstructured library")
except Exception as e:
    print("  ℹ️ Requires unstructured library with Excel support")


2️⃣ UnstructuredExcelLoader
  ℹ️ Requires unstructured library with Excel support


In [11]:
event_data = {
  "event_platform": {
    "platform_id": "EVT-PLT-001",
    "platform_name": "Atlas Events Hub",
    "events": [
      {
        "event_id": "EVT-1001",
        "event_name": "AI & Data Conference 2026",
        "category": "Technology",
        "date": {
          "start": "2026-04-10",
          "end": "2026-04-12"
        },
        "location": {
          "country": "Morocco",
          "city": "Marrakech",
          "venue": "Palais des Congrès"
        },
        "organizer": {
          "company_name": "Atlas Tech Group",
          "contact_email": "events@atlas-tech.com"
        },
        "participants": [
          {
            "participant_id": "P-001",
            "name": "Achraf Bogryn",
            "role": "Speaker",
            "ticket": {
              "type": "VIP",
              "price": 1500,
              "currency": "MAD"
            }
          },
          {
            "participant_id": "P-002",
            "name": "Sara El Idrissi",
            "role": "Attendee",
            "ticket": {
              "type": "Standard",
              "price": 500,
              "currency": "MAD"
            }
          }
        ],
        "sessions": [
          {
            "session_id": "S-01",
            "title": "Building Production-Ready ML Pipelines",
            "speaker": "Achraf Bogryn",
            "time": {
              "start": "10:00",
              "end": "11:30"
            }
          },
          {
            "session_id": "S-02",
            "title": "MLOps Best Practices",
            "speaker": "Sara El Idrissi",
            "time": {
              "start": "14:00",
              "end": "15:00"
            }
          }
        ]
      }
    ]
  }
}


In [ ]:
with open("data/json_files/event_data.json","w") as f:
    f.du